### 디코더에서 수행하는 셀프 어텐션
---  
다음 그림은 인코더와 디코더 블록을 나타낸 그림입니다. 그림에서도 확인할 수 있듯이 디코더 입력은 인코더 마지막 블록에서 나온 소스 단어 벡터 시퀀스(그림에서 붉은색 실선), 이전 디코더 블록의 수행 결과로 도출된 타깃 단어 벡터 시퀀스(그림에서 파란색 실선)입니다.

<center><그림 3-31 인코더-디코더></center>  

![](https://i.imgur.com/qaMh3TR.png)    



<center>출처 : ratsgo's NLPBOOK</center>

그러면 디코더에서 수행되는 셀프 어텐션을 순서대로 살펴보겠습니다. 우선 ***마스크 멀티 헤드 어텐션(Masked Multi-Head Attention)*** 입니다. 이 모듈에서느 타깃 언어의 단어 벡터 시퀀스를 계산 대상으로 합니다. 한국어를 영어로 번역하는 task를 수행하는 트랜스포머 모델이라면 여기서 계산되는 대상은 영어 단어 시퀀스가 됩니다.

이 파트에서는 입력 시퀀스가 타깃 언어(영어)로 바뀌었을뿐 인코더 쪽 셀프 어텐션과 크게 다를 바가 없습니다. 다음 그림은 쿼리가 **cafe**일 때 셀프 어텐션을 나타낸 것입니다.  
학습이 잘 되었다면 쿼리, 키로부터 계산한 소프트맥스 확률 가운데 장소를 지칭하는 대명서 **There**가 높은 값을 지닐 겁니다. 이 확률과 밸류 벡터를 가중합해서 셀프 어텐션 계산을 마칩니다.

<center><그림 3-32 타깃 문장의 셀프 어텐션></center>  

![](https://i.imgur.com/OKZn5tF.jpg)  



<center>출처 : ratsgo's NLPBOOK</center>

그다음은 멀티 헤드 어텐션입니다. 인코더와 디코더 쪽 정보를 모두 활용합니다. 인코더에서 넘어 온 정보는 소스 언어 문장(**어제 카페 갔었어 거기 사람 많더라**)의 단어 벡터 시퀀스입니다. 디코더 직전 블록에서 넘어온 정보는 타깃 언어 문장((s) **I went to the cafe yesterday There ...**)의 단어 벡터 시퀀스입니다. 전자를 키, 후자를 쿼리로 삼아 셀프 어텐션 계산을 수행합니다.

다음 그림은 쿼리 단어가 **cafe**인 셀프 어텐션 계산을 나타낸 것입니다. 학습이 잘 되었다면 쿼리(타깃 언어 문장), 키(소스 언어 문장)로부터 계산한 소프트맥스 확률 가운데 쿼리에 대응하는 해당 장소를 지칭하는 단어 **카페**가 높은 값을 지닐 겁니다. 이 확률과 밸류 벡터를 가중합해서 셀프 어텐션 계산을 마칩니다.

<center><그림 3-33 소스-타깃 문장 간 셀프 어텐션></center>  

![](https://i.imgur.com/MEIltBw.jpg)  



<center>출처 : ratsgo's NLPBOOK</center>

그런데 학습 과정에서는 약간의 트림을 씁니다. 트랜스포머 모델의 최종 출력은 타깃 시퀀스 각각에 대한 확률 분포인데요, 모델이 한국어를 영어로 번역하는 task를 수행하고 있다면 영어 문장의 다음 단어로 어떤 것이 적절할지에 관한 확률이 됩니다.

예를 들어 인코더에 **어제 카페 갔었어 거기 사람 많더라**가, 디코더에 (s) 가 입력된 상황이라면 트랜스포머 모델은 다음 영어 단어 **I**를 맞히도록 학습됩니다. 하지만 학습 과정에서 모델에 이번에 맞혀야 할 정답인 **I**를 알려주게 되면 학습하는 의미가 사라집니다.

따라서 정답을 포함한 타깃 시퀀스의 미래 정보를 셀프 어텐션 계산에서 제외(마스킹)하게 됩니다. 이 때문에 디코더 블록의 첫 번째 어텐션을 마스크 멀티 헤드 어텐션이라고 부릅니다.  
다음 그림과 같습니다. 마스킹은 소프트맥스 확률이 0이 되도록 하여, 밸류와의 가중합에서 해당 단어 정보들이 무시되게끔 하는 방식으로 수행됩니다.

<center><그림 3-34 학습 시 디코더에서 수행되는 셀프 어텐션(1)></center>  

![](https://i.imgur.com/YkCRfGT.jpg)  



<center>출처 : ratsgo's NLPBOOK</center>

이전 그림처럼 셀프 어텐션을 수행하면 디코더 마지막 블록 출력 벡터 가운데 (s) 에 해당하는 벡터에는 소스 문장 전체의 문맥적 관계성이 함축되어 있습니다. 트랜스포머 모델은 이 (s) 벡터를 가지고 **I**를 맞히도록 학습합니다. 다시 말해 정답 **I**에 관한 확률은 높이고 다른 단어들의 확률은 낮아지도록 합니다.

<center><그림 3-35 모델 업데이트(1)></center>  

![](https://i.imgur.com/3e5enKy.jpg)  



<center>출처 : ratsgo's NLPBOOK</center>

다음은 인코더에 **어제 카페 갔었어 거기 사람 많더라**가, 디코더에 (s) **I**가 입력된 상황입니다. 따라서 이때의 셀프 어텐션은 정답 단어 **I** 이후의 모든 타깃 언어 단어들을 모델이 보지 못하도록 하는 방식으로 수행됩니다.

<center><그림 3-36 학습 시 디코더에서 수행되는 셀프 어텐션(2)></center>  

![](https://i.imgur.com/YkCRfGT.jpg)  



<center>출처 : ratsgo's NLPBOOK</center>

디코더 마지막 블록의 **I** 벡터에는 소스 문장(**어제 ... 갔더라**)과 (s) **I** 사이의 문맥적 관계성이 녹아 있습니다. 트랜스포머 모델은 이 **I** 벡터를 가지고 **went**를 맞히도록 학습합니다. 다시 말해 정답 **went**에 관한 확률은 높이고 다른 단어들의 확률은 낮아지도록 합니다.

<center><그림 3-35 모델 업데이트(2)></center>  

![](https://i.imgur.com/oVpa5G7.jpg)  



<center>출처 : ratsgo's NLPBOOK</center>

다음 그림은 인코더에 **어제 카페 갔었어 거기 사람 많더라**가, 디코더에 (s) **I went**가 입력된 상황입니다. 따라서 이때의 셀프 어텐션은 정답 단어 **to** 이후의 모든 타깃 언어 단어들을 모델이 보지 못하도록 하는 방식으로 수행됩니다.

<center><그림 3-38 학습 시 디코더에서 수행되는 셀프 어텐션(3)></center>  

![](https://i.imgur.com/Ls5T3wh.jpg)  



<center>출처 : ratsgo's NLPBOOK</center>

디코더 마지막 블록의 **went** 벡터에는 소스 문장과 (s) **I went** 사이의 문맥적 관계성이 녹아있습니다. 트랜스포머 모델은 이 **went** 벡터를 가지고 **to**를 맞히도록 학습합니다. 다시 말해 정답 **to**에 관한 확률은 높이고 다른 단어들의 확률은 낮아지도록 합니다.

<center><그림 3-39 모델 업데이트(3)></center>  

![](https://i.imgur.com/0GEfdNw.jpg)  



<center>출처 : ratsgo's NLPBOOK</center>

트랜스포머 모델은 이런 방식으로 말뭉치 전체를 훑어가면서 반복 학습합니다. 학습을 마친 모델은 다음처럼 기계 번역을 수행(인퍼런스)합니다.  

> 1. 소스 언어(한국어) 문장을 인코더에 입력해 인코더 마지막 블록의 단어 벡터 시퀀스를 추출합니다.  
>
> 2. 인코더에서 넘어온 소스 언어 문장 정보와 디코더에 타깃 문장 시작을 알리는 스페셜 토큰 (s) 를 넣어서, 타깃 언어(영어)의 첫 번째 토큰을 생성합니다.  
>
> 3. 인코더 쪽에서 넘어온 소스 언어 문장 정보와 이전에 생성된 타깃 언어 토큰 시퀀스를 디코더에 넣어서 만든 정보로 타깃 언어의 다음 토큰을 생성합니다.  
>
> 4. 생성된 문장 길이가 충분하거나 문장 끝을 알리는 스페셜 토큰 (s) 가 나올 때까지 3번 과정을 반복합니다.

한편 (s) 는 보통 타깃 언어 문장 맨 마지막에 붙여서 학습합니다. 이 토큰이 나타났다는 것은 모델이 타깃 문장 생성을 마쳤다는 의미입니다.